In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

In [2]:
from NFconstants import N_nod, Beta, DIM
from transforms import t_get_O
from flows.NormalizingFlow import NormalizingFlow    
from flows.RGflows import RGflows 
from transforms import config_RG_masks
from setup import osc
from obs import G
normal_dist = torch.distributions.Normal(loc=torch.zeros(N_nod,2), scale=torch.ones(N_nod,2))

In [3]:
from flows.theta import ThetaNetwork
th = ThetaNetwork(
        N_nod//2 * DIM,
        N_nod//2 * DIM,
        6,
        N_nod,
        2,
        0,
    )
z = torch.ones((10,N_nod//2,DIM))
t,s = th(z)
print(t.shape)
print(s.shape)

x shape torch.Size([10, 64])
torch.Size([10, 32, 1])
torch.Size([10, 32, 1])


In [ ]:
from flows.Layers import AffineCouplingLayer
from transforms import get_split_masks
from transforms import get_pair_split_masks
z = torch.ones((10,N_nod,DIM))
aff = AffineCouplingLayer(th, split = get_pair_split_masks(N_nod), swap = 0)
x,lad = aff.g(z)
print(x.shape)
print(lad.shape)

In [ ]:
flows =  NormalizingFlow.configure_flows(n_flows = 10,
                                                     num_hidden = 8,
                                                     hidden_dim = N_nod, 
                                                     dim = N_nod, 
                                                     param_dim = 0, 
                                                     p_drop = 0,sys_dim = DIM)
nf = NormalizingFlow(flows = flows)
x,lad = nf.g(z)
print(x.shape)
print(lad.shape)

In [6]:
m=3
RGmasks=config_RG_masks(m,N_nod)

In [7]:
hidden_dim_dict1={8:8,16:16,32:32,64:64,128:128,256:256,512:512}
n_flows_dict1={8:4,16:6,32:8,64:10,128:12,256:14,512:16}
num_hidden_dict1={8:6,16:6,32:6,64:8,128:8,256:8,512:10}

In [8]:
rg = RGflows.configure_RG_model(RGmasks,n_flows_dict1,num_hidden_dict1,hidden_dim_dict1,grids_no_grad=0,p_drop=0.0,sys_dim=2)
rg.save("./weights/model_weights_osc.pth")

In [21]:
rg = RGflows.load_model("./weights/model_weights_osc.pth")
z = torch.ones((10,N_nod,dim))
x,lad = rg.g(z)
print(x.shape)
print(lad.shape)

torch.Size([10, 64, 2])
torch.Size([10])
